In [2]:
import pandas as pd

In [28]:
# read data
raw_2019_2020 = pd.read_csv("2019_2020.csv")
raw_2020_2021 = pd.read_csv("2020_2021.csv")
raw_2021_2022 = pd.read_csv("2021_2022.csv")
raw_2022_2023 = pd.read_csv("2022_2023.csv")

raw_2019_2020.shape

(380, 106)

In [29]:
# target columns
columns = ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee",
           "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR"]

# filter Arsenal games with these columns
arsenal_2019_2020 = raw_2019_2020.loc[(raw_2019_2020["HomeTeam"]=="Arsenal") | (raw_2019_2020["AwayTeam"]=="Arsenal"), columns]
arsenal_2020_2021 = raw_2020_2021.loc[(raw_2020_2021["HomeTeam"]=="Arsenal") | (raw_2020_2021["AwayTeam"]=="Arsenal"), columns]
arsenal_2021_2022 = raw_2021_2022.loc[(raw_2021_2022["HomeTeam"]=="Arsenal") | (raw_2021_2022["AwayTeam"]=="Arsenal"), columns]
arsenal_2022_2023 = raw_2022_2023.loc[(raw_2022_2023["HomeTeam"]=="Arsenal") | (raw_2022_2023["AwayTeam"]=="Arsenal"), columns]

arsenal_2019_2020.shape

(38, 22)

In [30]:
# reformat date
arsenal_2019_2020["Date"] = pd.to_datetime(arsenal_2019_2020["Date"], format="%d/%m/%Y")
arsenal_2020_2021["Date"] = pd.to_datetime(arsenal_2020_2021["Date"], format="%d/%m/%Y")
arsenal_2021_2022["Date"] = pd.to_datetime(arsenal_2021_2022["Date"], format="%d/%m/%Y")
arsenal_2022_2023["Date"] = pd.to_datetime(arsenal_2022_2023["Date"], format="%d/%m/%Y")

# filter Mikel Arteta
mikel_joined_on = "2019-12-26"
arsenal_2019_2020 = arsenal_2019_2020[arsenal_2019_2020["Date"] >= mikel_joined_on]

# added a season column
arsenal_2019_2020["season"] = "2019-2020"
arsenal_2020_2021["season"] = "2020-2021"
arsenal_2021_2022["season"] = "2021-2022"
arsenal_2022_2023["season"] = "2022-2023"

# aggregate data (ONLY UNDER MIKEL)
arsenal_data = pd.concat([arsenal_2019_2020, arsenal_2020_2021, arsenal_2021_2022, arsenal_2022_2023])

arsenal_data.shape

(110, 23)

In [31]:
# Create Home Column
# Yes -> Arsenal is Home Team.
# No -> Arsenal is Away Team.

arsenal_data["Home"] = None # initial
arsenal_data.loc[arsenal_data["HomeTeam"] == "Arsenal", "Home"] = "Yes"
arsenal_data.loc[arsenal_data["HomeTeam"] != "Arsenal", "Home"] = "No"

# Create Result(Half Result) Column
# Win -> Arsenal win the game without any regards to home or away status.
# Lose -> Arsenal lose the game.
# Draw -> Arsenal draw the game.
arsenal_data["HalfResult"] = "Draw" # initial
arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal") &
                 (arsenal_data["HTR"] == "H"), "HalfResult"] = "Win"
arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal") & 
                 (arsenal_data["HTR"] == "A"), "HalfResult"] = "Lose"
arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal") & 
                 (arsenal_data["HTR"] == "A"), "HalfResult"] = "Win"
arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal") & 
                 (arsenal_data["HTR"] == "H"), "HalfResult"] = "Lose"

# Create Result(Final Result) Column
# Win -> Arsenal win the game without any regards to home or away status.
# Lose -> Arsenal lose the game.
# Draw -> Arsenal draw the game.
arsenal_data["FinalResult"] = "Draw" # initial
arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal") & 
                 (arsenal_data["FTR"] == "H"), "FinalResult"] = "Win"
arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal") & 
                 (arsenal_data["FTR"] == "A"), "FinalResult"] = "Lose"
arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal") & 
                 (arsenal_data["FTR"] == "A"), "FinalResult"] = "Win"
arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal") & 
                 (arsenal_data["FTR"] == "H"), "FinalResult"] = "Lose"

# Create Point Column
# 3 -> 3 points for Arsenal win
# 1 -> 1 point for Arsenal draw
# 0 -> 0 point for Arsenal loss
arsenal_data["Point"] = 0
arsenal_data.loc[arsenal_data["FinalResult"] == "Win", "Point"] = 3
arsenal_data.loc[arsenal_data["FinalResult"] == "Draw", "Point"] = 1
arsenal_data.loc[arsenal_data["FinalResult"] == "Lose", "Point"] = 0

# Create Full-Time-Goals-Taken (FTGT) and 
# Full-Time-Goals-Conceded (FTGC) columns
arsenal_data["FTGT"] = 0 # initial
arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal"), 
                 "FTGT"] = arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal"), 
                                            "FTHG"]
arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal"), 
                 "FTGT"] = arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal"), 
                                            "FTAG"]
arsenal_data["FTGC"] = 0 # initial
arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal"), 
                 "FTGC"] = arsenal_data.loc[(arsenal_data["HomeTeam"] == "Arsenal"), 
                                            "FTAG"]
arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal"), 
                 "FTGC"] = arsenal_data.loc[(arsenal_data["AwayTeam"] == "Arsenal"), 
                                            "FTHG"]

In [27]:
arsenal_data.shape

(110, 28)

In [32]:
arsenal_data.head(1)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AY,HR,AR,season,Home,HalfResult,FinalResult,Point,FTGT,FTGC
181,2019-12-26,Bournemouth,Arsenal,1,1,D,1,0,H,S Attwell,...,4,0,0,2019-2020,No,Lose,Draw,1,1,1


In [36]:
# make a final sheet
arsenal_data.to_csv("arsenal_under_mikel.csv", index=False)

In [37]:
test = pd.read_csv("arsenal_under_mikel.csv")
test.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AY,HR,AR,season,Home,HalfResult,FinalResult,Point,FTGT,FTGC
0,2019-12-26,Bournemouth,Arsenal,1,1,D,1,0,H,S Attwell,...,4,0,0,2019-2020,No,Lose,Draw,1,1,1
1,2019-12-29,Arsenal,Chelsea,1,2,A,1,0,H,C Pawson,...,4,0,0,2019-2020,Yes,Win,Lose,0,1,2
2,2020-01-01,Arsenal,Man United,2,0,H,2,0,H,C Kavanagh,...,0,0,0,2019-2020,Yes,Win,Win,3,2,0
3,2020-01-11,Crystal Palace,Arsenal,1,1,D,0,1,A,P Tierney,...,3,0,1,2019-2020,No,Win,Draw,1,1,1
4,2020-01-18,Arsenal,Sheffield United,1,1,D,1,0,H,M Dean,...,2,0,0,2019-2020,Yes,Win,Draw,1,1,1


In [38]:
test.shape

(110, 29)